# Parameters

In [1]:
from pathlib import Path

# DATA_PATH = 'data/'
DATA_PATH = "hf://datasets/INDEEPA/clip-siamese/"

In [2]:
SHARD_IDX = 0
TOTAL_SHARDS = 4
MAX_WORKERS = 2
UPLOAD_INTERVAL = 60 * 1

# Log into services

In [3]:
try:
    import dotenv
except ImportError:
    !pip install python-dotenv

In [4]:
# Use tokens from .env

import os
from dotenv import load_dotenv

import huggingface_hub
import wandb

load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")
huggingface_hub.login(token=HF_TOKEN)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# Load full data & take shard

In [5]:
import pandas as pd


file_path = DATA_PATH + 'Карты_мира_озон.xlsm'
full_source_df = pd.read_excel(file_path)
full_source_df.head(1)

,SKU,Name,Category,Схема,Brand,Niche,Seller,Balance,Balance FBS,Warehouses count,...,FBS,Base price,Category Position,Categories Last Count,Sales Per Day Average,Turnover,Frozen stocks,Frozen stocks cost,Frozen stocks percent,is_primary
0,936454663,Карта мира географическая политическая интерак...,Канцелярские товары/Бумажная продукция/Карты мира,FBO,NaN,Дом и сад/Декор и интерьер/Картины и панно/Гра...,GooDaY,346,0,12,...,0,4990,1,1,5.2,66.54,112,202832,32.37,False


In [6]:
import pandas as pd
from sklearn.model_selection import KFold

def get_shard(full_source_df, total_shards, shard_idx, seed=42):
    """
    Splits the full_source_df into `total_shards` folds using a fixed seed,
    and returns the fold corresponding to `shard_idx`.
    
    Args:
        full_source_df (pd.DataFrame): The input DataFrame to be split.
        total_shards (int): Total number of folds.
        shard_idx (int): Index of the fold to select (0-indexed).
        seed (int): Random seed for reproducibility (default: 42).
    
    Returns:
        pd.DataFrame: The fold corresponding to `shard_idx`.
    """
    if total_shards > 1:
        kf = KFold(n_splits=total_shards, shuffle=True, random_state=seed)
        # Create list of (train_index, test_index) pairs
        folds = list(kf.split(full_source_df))
        # Select the test indices from the desired fold
        _, test_indices = folds[shard_idx]
        source_df = full_source_df.iloc[test_indices]
    else:
        source_df = full_source_df

    return source_df

# Assuming source_df is your DataFrame
source_df = get_shard(full_source_df, TOTAL_SHARDS, SHARD_IDX)
source_df.head()

,SKU,Name,Category,Схема,Brand,Niche,Seller,Balance,Balance FBS,Warehouses count,...,FBS,Base price,Category Position,Categories Last Count,Sales Per Day Average,Turnover,Frozen stocks,Frozen stocks cost,Frozen stocks percent,is_primary
8,1712946829,"Карта мира физическая на стену, географическая...",Канцелярские товары/Бумажная продукция/Карты мира,FBO,Brauberg,Дом и сад/Декор и интерьер/Картины и панно/Гра...,Самсон,120,0,6,...,0,3057,5,1,3.33,36.00,0,0,0.00,False
12,1713026634,Карта мира политическая на стену большая 157х1...,Канцелярские товары/Бумажная продукция/Карты мира,FBO,Brauberg,Дом и сад/Декор и интерьер/Картины и панно/Гра...,Самсон,83,0,5,...,0,1870,43,1,4.37,19.01,0,0,0.00,False
15,950549378,"Карта Мира настенная, с государственными флага...",Канцелярские товары/Бумажная продукция/Карты мира,FBO,Карта ТД,Дом и сад/Декор и интерьер/Картины и панно/Гра...,КАРТА,34,0,10,...,0,4199,247,1,1.80,18.89,0,0,0.00,False
17,491270369,Географическая карта России настенная 102х160 ...,Канцелярские товары/Бумажная продукция/Карты мира,FBO,Атлас Принт,Дом и сад/Декор и интерьер/Картины и панно/Гра...,ИНТЕРТРЕЙД,102,797,8,...,1,2242,32,1,3.90,26.15,0,0,0.00,True
23,1422369500,Скретч карта мира Летопись Путешествий СЕРЕБРО...,Канцелярские товары/Бумажная продукция/Карты мира,FBO,Правила Успеха,Дом и сад/Декор и интерьер/Картины и панно/Гра...,Правила Успеха Упаковки,107,172,10,...,1,3800,95,1,0.80,133.75,71,216124,66.36,False


# Scraping

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium_stealth import stealth

# Scraping Browser

In [8]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1920, 1080))

display.start()
# display.stop()

In [9]:
# CELL #1: Init driver definition

import random
from seleniumwire import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium_stealth import stealth

# Use rotating proxy endpoint credentials (instead of a random proxy from a list)
proxy_host = "p.webshare.io"
proxy_port = "80"
proxy_user = "ezazhhrq-rotate"
proxy_pass = "1cacuvqw8gcx"

# Configure selenium-wire proxy options with rotating credentials
proxy_options = {
    'proxy': {
        'http': f'http://{proxy_user}:{proxy_pass}@{proxy_host}:{proxy_port}',
        'https': f'http://{proxy_user}:{proxy_pass}@{proxy_host}:{proxy_port}'
    },
    'disable_socks_interceptor': True  # Use if auth popup appears
}


# Set up Chrome options
options = Options()
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)
# Uncomment for headless mode if desired:
options.add_argument("--headless")
options.add_argument("--window-size=960,1080")
options.add_argument("--window-position=960,0")       # Start at x=960 (right half of a 1920px screen)
options.add_argument("--disable-blink-features=AutomationControlled")

import random
import uuid
import tempfile

import random

def generate_user_agent():
    """
    Randomly selects and returns one user agent string from a fixed list.
    """
    agents = [
        {"ua": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.3"},
        {"ua": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36 Trailer/93.3.8652.5"},
        {"ua": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36 Edg/134.0.0."},
        {"ua": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36 OPR/117.0.0."},
        {"ua": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.1958"},

        # {"ua": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.10 Safari/605.1.1"},
        # {"ua": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.3"},
        # {"ua": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.3"},
        # {"ua": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0."},
        # {"ua": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36 Edg/132.0.0."},
        # {"ua": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:136.0) Gecko/20100101 Firefox/136."},
        # {"ua": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.3"}
    ]
    return random.choice(agents)["ua"]

def init_scraping_browser(use_proxy=True, verbose=True):
    # Generate a unique profile directory for this driver session
    unique_suffix = uuid.uuid4().hex
    unique_profile = tempfile.mkdtemp(prefix=f"chrome_profile_{unique_suffix}_")
    
    # Create Chrome options and add the unique profile directory
    options = webdriver.ChromeOptions()
    options.add_argument(f"--user-data-dir={unique_profile}")
    
    # Set a dynamically generated user agent
    ua = generate_user_agent()
    options.add_argument(f"--user-agent={ua}")
    
    # (Optional) Uncomment to run headless
    options.add_argument("--headless")
    
    # Prepare additional driver kwargs if necessary
    driver_kwargs = dict(
        service=ChromeService(ChromeDriverManager().install()),
        options=options,
    )

    if use_proxy:
        driver_kwargs.update({
            'seleniumwire_options': proxy_options
        })

    driver = webdriver.Chrome(**driver_kwargs)
    
    # Generate random stealth options
    languages_options = ["en-US", "en", "fr-FR", "es-ES", "de-DE"]
    platform_options = ["Win32", "Linux x86_64", "MacIntel"]
    webgl_vendor_options = ["Intel Inc.", "NVIDIA Corporation", "AMD", "Google Inc."]
    renderer_options = [
        "Intel Iris OpenGL Engine",
        "ANGLE (NVIDIA GeForce GTX 1050 Ti Direct3D11 vs_5_0)",
        "ANGLE (Intel HD Graphics 520 Direct3D11 vs_5_0)"
    ]
    
    random_stealth_options = {
        "languages": [random.choice(languages_options)],
        "vendor": random.choice(["Google Inc.", "Microsoft Corporation", "Apple Inc."]),
        "platform": random.choice(platform_options),
        "webgl_vendor": random.choice(webgl_vendor_options),
        "renderer": random.choice(renderer_options),
        "fix_hairline": True
    }
    
    # Apply stealth settings to the driver (assuming stealth is set up)
    stealth(driver,
            languages=random_stealth_options["languages"],
            vendor=random_stealth_options["vendor"],
            platform=random_stealth_options["platform"],
            webgl_vendor=random_stealth_options["webgl_vendor"],
            renderer=random_stealth_options["renderer"],
            fix_hairline=random_stealth_options["fix_hairline"])
    
    if verbose:
        print(f"[DEBUG] New driver session id: {driver.session_id} using profile: {unique_profile}")
        print(f"[DEBUG] User Agent: {ua}")

    return driver

# Example usage:
# driver = init_scraping_browser()


In [10]:
from selenium.webdriver.common.by import By
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Define a custom exception
class AntiBotException(Exception):
    """Raised when a reload button is detected, indicating the page requires reloading."""
    pass

def extract_description(driver, sample_url, verbose=False, timeout=8):
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.common.exceptions import TimeoutException

    if verbose:
        print(f"[INFO] NAVIGATING TO (timeout={timeout}): {sample_url}")
    driver.get(sample_url)

    # Wait until one of the four elements is present:
    # - description element (class "k7y_28")
    # - alternative description element (class "s0k_28")
    # - reload button (id "reload-button")
    # - out-of-stock indicator (a element with class "v1l_28")
    WebDriverWait(driver, timeout).until(
        lambda d: d.find_elements(By.CLASS_NAME, "k7y_28") or 
                  d.find_elements(By.CLASS_NAME, "s0k_28") or
                  d.find_elements(By.ID, "reload-button") or 
                  d.find_elements(By.CSS_SELECTOR, "a.v1l_28")
    )

    # Check for a reload button first; if found, raise AntiBotException.
    if driver.find_elements(By.ID, "reload-button"):
        raise AntiBotException(f"Reload button detected on {sample_url}; page requires reloading.")

    # If an out-of-stock indicator is found, log and return None.
    if driver.find_elements(By.CSS_SELECTOR, "a.v1l_28"):
        if verbose:
            print("[INFO] OUT-OF-STOCK INDICATOR DETECTED. IGNORING")
        return None

    # Attempt to retrieve the description from element with class "k7y_28"
    description_elements = driver.find_elements(By.CLASS_NAME, "k7y_28")
    if description_elements:
        if verbose:
            print("[INFO] DESCRIPTION DETECTED (k7y_28)")
        return description_elements[0].text.strip()

    # If not found, fall back to the alternative description element "s0k_28"
    description_elements = driver.find_elements(By.CLASS_NAME, "s0k_28")
    if description_elements:
        if verbose:
            print("[INFO] DESCRIPTION EMPTY (s0k_28)")
        return None

    if verbose:
        print("[INFO] DESCRIPTION ELEMENT NOT FOUND")
    return None

# out_of_stock_indices = [
#     4009,
#     1388,
#     1446,
#     5696
# ]

# empty_description_indices = [
#     3959
# ]

# # Example usage:
# driver = init_scraping_browser()
# sample_url = source_df.sample(1)['URL'].item()
# # sample_url = source_df.loc[out_of_stock_indices].sample(1)['URL'].item()
# # sample_url = source_df.loc[empty_description_indices].sample(1)['URL'].item()
# try:
#     description = extract_description(
#         driver, sample_url, verbose=True, timeout=20
#     )
#     print(f"\nDescription: {description}")
# except AntiBotException as e:
#     print(f"\nBlocked: {e}")

In [11]:
# CELL #4: Retry wrapper

TIMEOUT_WITH_PROXY = 20

def retry_extract_description(
    driver, sample_url, verbose=False,
    timeout=20, max_retries=10, timeout_delta=2
):
    """
    Tries to extract a description from sample_url using extract_description().
    On AntiBotException, it quits the current driver and reinitializes a fresh one.
    Retries until max_retries is reached, then raises TimeoutException.
    
    Returns:
        description (str): The extracted description text.
    """
    from selenium.common.exceptions import TimeoutException
    attempt = 0
    while attempt < max_retries:
        attempt += 1
        if verbose:
            print(f"[RETRY WRAPPER] (ATTEMPT {attempt} | timeout {timeout}) - Using driver id: {id(driver)}")
        try:
            description = extract_description(driver, sample_url, verbose=verbose, timeout=timeout)
            driver.quit()
            return description
        except AntiBotException as e:
            if verbose:
                print(f"[RETRY WRAPPER] (ATTEMPT {attempt} | timeout {timeout}) - AntiBotException detected: {e}. Reinitializing driver.")
            driver.quit()
            driver = init_scraping_browser(use_proxy=True, verbose=verbose)
            timeout = TIMEOUT_WITH_PROXY
        except TimeoutException as e:
            if verbose:
                print(f"[RETRY WRAPPER] (ATTEMPT {attempt} | timeout {timeout}) - Timeout. Increasing by {timeout_delta}")
            driver.quit()
            driver = init_scraping_browser(use_proxy=True, verbose=verbose)
            timeout += timeout_delta
        except Exception as ex:
            if verbose:
                print(f"[RETRY WRAPPER] (ATTEMPT {attempt} | timeout {timeout}) - ERROR: {ex}. Retrying.")
        import time
        time.sleep(1)
    raise TimeoutException(f"Max retries reached for {sample_url}")


# # Initialize a driver instance.
# driver = init_scraping_browser()
# sample_url = source_df.sample(1)['URL'].item()
# try:
#     description = retry_extract_description(
#         driver, sample_url, verbose=True, 
#         timeout=20, max_retries=10, timeout_delta=2
#     )

#     print(f"\nDescription: {description}")
# except AntiBotException as e:
#     print(f"\nAntibot page detected: {e}")
# except Exception as e:
#     print(f"\nOther error: {e}")
# finally:
#     driver.quit()

In [12]:
# CELL #5: Parallel writing to single .parquet file

import concurrent
import numpy as np
import threading
from queue import Queue
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import nest_asyncio
nest_asyncio.apply()

# ---------------------------------------------------------------------------
# Assume source_df is your DataFrame with a 'URL' column.
# Also assume that init_scraping_browser() and extract_img_link_and_descr()
# are defined (the latter accepts parameters: driver, sample_url, row_id, verbose).

# ---------------------------------------------------------------------------
def writer_thread_func(result_queue, output_file):
    schema = pa.schema([
        ('SKU', pa.string()),
        ('URL', pa.string()),
        ('description', pa.string()),
    ])
    writer = None
    while True:
        result = result_queue.get()
        if result is None:
            result_queue.task_done()
            break  # Sentinel value to exit.
        sku, url, description = result
        data = {
            'SKU': [str(sku)],
            'URL': [str(url)],
            'description': [str(description) if description is not None else None],
        }
        try:
            table = pa.Table.from_pydict(data, schema=schema)
        except Exception as ex:
            print(f"Data conversion error for SKU {sku}: {data} -> {ex}")
            result_queue.task_done()
            continue
        if writer is None:
            writer = pq.ParquetWriter(output_file, schema)
        writer.write_table(table)
        result_queue.task_done()
    if writer is not None:
        writer.close()

In [ ]:
from pathlib import Path
from huggingface_hub import HfApi, HfFolder

def upload_file_to_hf(output_file, repo_id="INDEEPA/clip-siamese", repo_type="dataset", verbose=True):
    """
    Uploads a file to a Hugging Face repository using HfApi.upload_file.
    
    The remote file path is constructed from the file's immediate parent folder 
    and its file name. For example, if output_file is:
      "tmp/OZ_geo_5500_descriptions/part.0.parquet"
    then the file is uploaded as:
      "OZ_geo_5500_descriptions/part.0.parquet"
    
    Args:
        output_file (str): The path to the local file.
        repo_id (str): The Hugging Face repository ID (default is "INDEEPA/clip-siamese").
        repo_type (str): The type of repository (default is "dataset").
    """
    # Retrieve your Hugging Face token (ensure it's set in your environment)
    token = HfFolder.get_token()
    
    # Create a Path object for easier manipulation
    path_obj = Path(output_file)
    
    # Extract the immediate parent folder name and the file name
    parent_folder = path_obj.parent.name
    file_name = path_obj.name
    
    # Construct the remote file path using both parent folder and file name
    remote_file_path = f"{parent_folder}/{file_name}"
    
    # Upload the file using HfApi.upload_file
    HfApi().upload_file(
        path_or_fileobj=output_file,
        path_in_repo=remote_file_path,
        repo_id=repo_id,
        repo_type=repo_type,
        token=token,
    )
    
    if verbose:
      print(f"Uploaded {output_file} as {remote_file_path} to {repo_id}")

# # Example usage:
# upload_file_to_hf(f'tmp/OZ_geo_5500_descriptions/part.{SHARD_IDX}.parquet')

In [ ]:
# With periodic uploading

import os
import threading
import time
from queue import Queue
import concurrent.futures
from pathlib import Path
from tqdm import tqdm
from huggingface_hub import HfApi, HfFolder

def process_task(sku, url, verbose, timeout, max_retries):
    driver = init_scraping_browser(verbose=verbose)  # Create a fresh driver instance.
    try:
        # retry_extract_description() handles retries on its own.
        description = retry_extract_description(driver, url, verbose=verbose, timeout=timeout, max_retries=max_retries)
        return sku, url, description
    finally:
        driver.quit()

def main(
    source_df,
    max_workers=2,
    verbose=True,
    file_dir='tmp',
    output_file_name="OZ_geo_5500_descriptions",
    part_idx=0,
    upload_interval=60  # time interval in seconds between uploads
):
    result_queue = Queue()
    # Create the folder structure only if not using an hf:// path.
    if not str(file_dir).startswith('hf'):
        os.makedirs(os.path.join(file_dir, output_file_name), exist_ok=True)

    output_file_path = os.path.join(file_dir, output_file_name, f'part.{part_idx}.parquet')

    # Start the writer thread that writes results to the parquet file.
    writer_thread = threading.Thread(target=writer_thread_func, args=(result_queue, output_file_path))
    writer_thread.start()

    # Start the periodic uploader thread.
    stop_event = threading.Event()
    def periodic_uploader():
        while not stop_event.is_set():
            upload_file_to_hf(output_file_path, verbose=verbose)
            # Wait for the interval or until stop_event is set.
            stop_event.wait(upload_interval)
    uploader_thread = threading.Thread(target=periodic_uploader)
    uploader_thread.start()

    # Create a dictionary mapping SKU to URL (assuming source_df has columns "SKU" and "URL").
    urls_dict = source_df.set_index("SKU")["URL"].to_dict()

    # Use ProcessPoolExecutor to process each SKU/URL pair concurrently in isolated processes.
    with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
        future_to_sku = {
            executor.submit(process_task, sku, url, verbose, 20, 10): sku 
            for sku, url in urls_dict.items()
        }
        # Wrap the as_completed iterator with tqdm to show progress.
        for future in tqdm(concurrent.futures.as_completed(future_to_sku),
                           total=len(future_to_sku),
                           desc="Processing tasks"):
            sku = future_to_sku[future]
            try:
                sku, url, description = future.result()
                if verbose:
                    if description is not None:
                        print(f"[{sku}] - DESCRIPTION SUCCESS: {description[:60]}...")
                    else:
                        print(f"[{sku}] - DESCRIPTION OMITTED (OOS)")
                result_queue.put((sku, url, description))
            except Exception as e:
                print(f"[{sku}] - ERROR {type(e).__name__} PROCESSING ROW: {e}")

    # Wait until all queued results are written.
    result_queue.join()
    result_queue.put(None)
    writer_thread.join()

    # Signal the uploader thread to stop and wait for it to finish.
    stop_event.set()
    uploader_thread.join()

    # Optionally, perform one final upload.
    upload_file_to_hf(output_file_path, verbose=verbose)
    print("Scraping completed and descriptions appended to:", output_file_path)

out_of_stock_indices = [
    4009,
    1388,
    1446,
    5696
]

empty_description_indices = [
    3959
]

# ---------------------------------------------------------------------------
# Run main with a sample of source_df entries.
main(
    source_df=source_df,
    # source_df=source_df.sample(4),
    # source_df=source_df.loc[out_of_stock_indices],
    # source_df=source_df.loc[empty_description_indices],
    max_workers=MAX_WORKERS,
    # verbose=False,
    # verbose=True,
    part_idx=SHARD_IDX,
    upload_interval=UPLOAD_INTERVAL,
    
    # file_dir=DATA_PATH,
)

Processing tasks:   0%|          | 0/1426 [00:00<?, ?it/s]No files have been modified since last commit. Skipping to prevent empty commit.


[DEBUG] New driver session id: b06ed8fd7f81cc5e9698403370b8f2d0 using profile: /tmp/chrome_profile_448fbd941bea4177af4846f84d2d65ba_u503wzen
[DEBUG] User Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.3
[RETRY WRAPPER] (ATTEMPT 1 | timeout 20) - Using driver id: 139636430808336
[INFO] NAVIGATING TO (timeout=20): https://www.ozon.ru/context/detail/id/1712946829/
[DEBUG] New driver session id: 0c6d0fc02096fa4b9fb0078189c2608b using profile: /tmp/chrome_profile_84682629655e4219af90c9967fea3e4f_5m6wosup
[DEBUG] User Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.1958
[RETRY WRAPPER] (ATTEMPT 1 | timeout 20) - Using driver id: 139636430808672
[INFO] NAVIGATING TO (timeout=20): https://www.ozon.ru/context/detail/id/1713026634/
Uploaded tmp/OZ_geo_5k_descriptions/part.0.parquet as OZ_geo_5k_descriptions/part.0.parquet to INDEEPA/clip-siamese


Processing tasks:   0%|          | 1/1426 [00:30<12:10:38, 30.76s/it]

[1712946829] - DESCRIPTION SUCCESS: Вас когда-нибудь завораживала идея путешествий? Настенная ка...
[DEBUG] New driver session id: 2fb12fffd5648ecc7773a700fac0ee9d using profile: /tmp/chrome_profile_14ee081102514bc3a06d0f8acfc018a5_nza1kj7n
[DEBUG] User Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.1958
[RETRY WRAPPER] (ATTEMPT 1 | timeout 20) - Using driver id: 139637609082832
[INFO] NAVIGATING TO (timeout=20): https://www.ozon.ru/context/detail/id/950549378/
[RETRY WRAPPER] (ATTEMPT 1 | timeout 20) - AntiBotException detected: Reload button detected on https://www.ozon.ru/context/detail/id/950549378/; page requires reloading.. Reinitializing driver.
[DEBUG] New driver session id: 620e63730320bffd33a863e3e2bc9a49 using profile: /tmp/chrome_profile_aefc66f2318046e9b5da981d7bf570e5_s553h1hy
[DEBUG] User Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.

Processing tasks:   0%|          | 2/1426 [00:39<6:57:55, 17.61s/it] 

...

[1825850417] - DESCRIPTION SUCCESS: Учебная топографическая карта У-34-37 (ЯСНОГОРСК)

Масштаб 1...
[DEBUG] New driver session id: 81bbe3ffafd03dd24bedefdb61d66f4d using profile: /tmp/chrome_profile_03c549c31e2f4935b018809871412fe3_av5cumtm
[DEBUG] User Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.1958
[RETRY WRAPPER] (ATTEMPT 1 | timeout 20) - Using driver id: 139636373363120
[INFO] NAVIGATING TO (timeout=20): https://www.ozon.ru/context/detail/id/1165370447/
[RETRY WRAPPER] (ATTEMPT 1 | timeout 20) - AntiBotException detected: Reload button detected on https://www.ozon.ru/context/detail/id/1165370447/; page requires reloading.. Reinitializing driver.
[DEBUG] New driver session id: 211e97e3fbd86e6adabac2105a497bc8 using profile: /tmp/chrome_profile_0301f130a4904b45a1e33eaa6415102c_rcl6fmfr
[DEBUG] User Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.

Processing tasks:  37%|███▋      | 532/1426 [4:02:19<4:33:00, 18.32s/it]

[1207346537] - DESCRIPTION SUCCESS: Скретч карта России - отличный подарок патриоту и всем, кто ...
[DEBUG] New driver session id: f299f4f19c4fb2133ff57a375bc6f457 using profile: /tmp/chrome_profile_473fd842a9eb490eafe1a08e72236a38_glites70
[DEBUG] User Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.1958
[RETRY WRAPPER] (ATTEMPT 1 | timeout 20) - Using driver id: 139636370046128
[INFO] NAVIGATING TO (timeout=20): https://www.ozon.ru/context/detail/id/1031758727/
[INFO] DESCRIPTION DETECTED (k7y_28)


Processing tasks:  37%|███▋      | 533/1426 [4:02:48<5:20:48, 21.55s/it]

[1165370447] - DESCRIPTION SUCCESS: Двухсторонняя покрытая полимерной плёнкой карта имеет размер...
[RETRY WRAPPER] (ATTEMPT 1 | timeout 20) - Timeout. Increasing by 2
[DEBUG] New driver session id: 4cf4d1271cb98fbee9451c958d36bde4 using profile: /tmp/chrome_profile_bd362c8cef5d4da2a03713f7c60152fe_n_16wibr
[DEBUG] User Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.3
[DEBUG] New driver session id: 5f1f54ac06edad9efa25ef2590df08e4 using profile: /tmp/chrome_profile_c4ce6cf6f6ef4d09bf6f813402344272_gsxixxxh
[DEBUG] User Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36 OPR/117.0.0.
[RETRY WRAPPER] (ATTEMPT 1 | timeout 20) - Using driver id: 139637616073584
[INFO] NAVIGATING TO (timeout=20): https://www.ozon.ru/context/detail/id/679323707/
[RETRY WRAPPER] (ATTEMPT 2 | timeout 22) - Using driver id: 139637612680976
[INFO] NAVIGATING TO (timeout=22): htt

In [ ]:
# # CELL 6: Running the main function

# def process_task(sku, url, verbose, timeout, max_retries):
#     driver = init_scraping_browser(verbose=verbose)  # Create a fresh driver instance.
#     try:
#         # retry_extract_description() handles retries on its own.
#         description = retry_extract_description(driver, url, verbose=verbose, timeout=timeout, max_retries=max_retries)
#         return sku, url, description
#     finally:
#         driver.quit()

# # CELL #6: Running the main function using ProcessPoolExecutor with tqdm progress bar

# def main(
#     source_df,
#     max_workers=2,
#     verbose=True,
#     file_dir='tmp',
#     output_file_name = "OZ_geo_5500_descriptions",
#     part_idx = 0
# ):
#     from queue import Queue
#     import concurrent.futures
#     import threading
#     import os
#     from tqdm import tqdm

#     result_queue = Queue()
#     if not str(file_dir).startswith('hf'):
#         os.makedirs(file_dir + '/' + output_file_name, exist_ok=True)

#     output_file_path = file_dir + '/' + output_file_name + f'/part.{part_idx}.parquet'

#     # Start the writer thread that writes results to a parquet file.
#     writer_thread = threading.Thread(target=writer_thread_func, args=(result_queue, output_file_path))
#     writer_thread.start()

#     # Create a dictionary mapping SKU to URL (assuming source_df has columns "SKU" and "URL").
#     urls_dict = source_df.set_index("SKU")["URL"].to_dict()

#     # Use ProcessPoolExecutor to process each SKU/URL pair concurrently in isolated processes.
#     with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
#         future_to_sku = {
#             executor.submit(process_task, sku, url, verbose, 20, 10): sku 
#             for sku, url in urls_dict.items()
#         }
#         # Wrap the as_completed iterator with tqdm to show progress.
#         for future in tqdm(concurrent.futures.as_completed(future_to_sku),
#                            total=len(future_to_sku),
#                            desc="Processing tasks"):
#             sku = future_to_sku[future]
#             try:
#                 sku, url, description = future.result()
#                 if verbose:
#                     if description is not None:
#                         print(f"[{sku}] - DESCRIPTION SUCCESS: {description[:60]}...")
#                     else:
#                         print(f"[{sku}] - DESCRIPTION OMITTED (OOS)")
#                 result_queue.put((sku, url, description))
#             except Exception as e:
#                 print(f"[{sku}] - ERROR {type(e).__name__} PROCESSING ROW: {e}")

#     # Wait until all queued results are written.
#     result_queue.join()
#     result_queue.put(None)
#     writer_thread.join()

#     upload_file_to_hf(output_file_path)
#     print("Scraping completed and descriptions appended to:", output_file_path)


# out_of_stock_indices = [
#     4009,
#     1388,
#     1446,
#     5696
# ]

# empty_description_indices = [
#     3959
# ]

# # ---------------------------------------------------------------------------
# # Run main with a sample of source_df entries.
# main(
#     # source_df=source_df,
#     source_df=source_df.sample(4),
#     # source_df=source_df.loc[out_of_stock_indices],
#     # source_df=source_df.loc[empty_description_indices],
#     max_workers=2,
#     verbose=False,
#     part_idx=SHARD_IDX
    
#     # file_dir=DATA_PATH,
# )

In [ ]:
# df = pd.read_parquet(f'tmp/OZ_geo_5500_descriptions/part.{SHARD_IDX}.parquet')
# df.shape

(4, 3)